In [19]:
#C:\Users\pekas\Downloads\Bilder Wuerfel 2 small train test

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import ReduceLROnPlateau


In [ ]:
train_data = keras.utils.image_dataset_from_directory(
    r'C:\Users\pekas\Downloads\Bilder Wuerfel 2 small train test\Bilder Wuerfel 2 small train',  # Pfad zu den Bildern
    labels='inferred',            # Labels aus den Dateiname ableiten
    # 'int' oder 'categorical' je nach Aufgabe Regression oder Klassifikation
    label_mode='int',
    class_names=None,             # Man kann optional eine Liste der Klassennamen übergeben
    color_mode='rgb',             # Bilder farbig ausgeben. Alternativ: 'grayscale', 'rgba'
    # Wie viele Bilder auf einmal von der Festplatte geladen werden
    batch_size=1,
    image_size=(128, 128),        # Biler auf die angegebene Größe sklaieren
    shuffle=True,                 # Vor jeder Epoche die Bilder durchmischen
    # if not None Durchmischen in bestimmter Reihenfolge durchführen
    seed=0,
    validation_split=None,         # Gibt den Anteil der Validationdaten an
    # Gibt an, ob man die Trainigs- oder Validationdaten bekommen möchte
    subset=None,
    interpolation='bilinear',     # Wie die Skalierung durchfgeführt wird
    follow_links=False,           # Ordnerstruktur nachverfolgen
    # Auf die Bildmitte zuschneiden um das Seitenverhältnis nicht zu verändern
    crop_to_aspect_ratio=False)

In [ ]:
validation_data = keras.utils.image_dataset_from_directory(
    r'C:\Users\pekas\Downloads\Bilder Wuerfel 2 small train test\Bilder Wuerfel 2 small test',  # Pfad zu den Bildern
    labels='inferred',            # Labels aus den Dateiname ableiten
    # 'int' oder 'categorical' je nach Aufgabe Regression oder Klassifikation
    label_mode='int',
    class_names=None,             # Man kann optional eine Liste der Klassennamen übergeben
    color_mode='rgb',             # Bilder farbig ausgeben. Alternativ: 'grayscale', 'rgba'
    # Wie viele Bilder auf einmal von der Festplatte geladen werden
    batch_size=1,
    image_size=(128, 128),        # Biler auf die angegebene Größe sklaieren
    shuffle=True,                 # Vor jeder Epoche die Bilder durchmischen
    # if not None Durchmischen in bestimmter Reihenfolge durchführen
    seed=None,
    validation_split=None,        # Gibt den Anteil der Validationdaten an
    # Gibt an, ob man die Trainigs- oder Validationdaten bekommen möchte
    subset=None,
    interpolation='bilinear',     # Wie die Skalierung durchfgeführt wird
    follow_links=False,           # Ordnerstruktur nachverfolgen
    # Auf die Bildmitte zuschneiden um das Seitenverhältnis nicht zu verändern
    crop_to_aspect_ratio=False)

In [ ]:
image=[]
label=[]
for images, labels in train_data:
    image.append(images[0])
    label.append(labels[0])

In [ ]:
X_train=np.array(image)
y_train=np.array(label)
y_train=y_train+1
X_train.shape

In [ ]:
def adjust_red(x, training):
    if training:
        x = np.array(x)
        x[:, :, :, 0] = x[:, :, :, 0] + \
            np.random.random(x[:, :, :, 0].shape)*128
        x = np.clip(x, 0, 255)
    return x


In [ ]:
labels, counts = np.unique(y_train, return_counts=True)
labels

In [ ]:
X_train = tf.image.resize(X_train, [224, 224])
X_val = tf.image.resize(X_val, [224, 224])
X_train.shape

In [ ]:
indices_train= np.arange(len(X_train))  # Créer une liste d'indices
indices_train=np.random.shuffle(indices_train)  # Mélanger les indices
indices_val = np.arange(len(X_val))  # Créer une liste d'indices
indices_val=np.random.shuffle(indices_val)  # Mélanger les indices

In [ ]:
X_train =X_train[indices_train][0]
y_train = y_train[indices_train][0]
X_train.shape

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Input((224, 224, 3)))
model.add(keras.layers.RandomRotation(
    0.02,  # 0.02 * 2*pi = 7.2°
    fill_mode="constant",
    interpolation="bilinear",
    seed=0,
    fill_value=128,
    input_shape=(128, 128, 3)))
model.add(keras.layers.RandomZoom(
    [-0.2, 0.0],  # Nur in das Bild hinenzoomen
    # width_factor=[-0.2, 0],
    fill_mode="nearest",
    interpolation="bilinear",
    seed=0,
    fill_value=180))
model.add(keras.layers.RandomFlip(
    mode="horizontal", seed=0))
model.add(keras.layers.RandomContrast(0.2, seed=0))
model.add(keras.layers.RandomBrightness([-0.3, 0.6],
                                        value_range=(0, 255),
                                        seed=0
                                        ))
model.add(tf.keras.applications.MobileNet(
    weights='imagenet',
    input_shape=(224, 224, 3),
    classes=1000,
    include_top=False,
    pooling='avg'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='linear'))
model.summary()

In [ ]:
model.compile(loss="mse",optimizer=keras.optimizers.Adam(learning_rate=0.001),metrics=[tfa.metrics.RSquare()
])

In [ ]:
lr_callback = ReduceLROnPlateau(monitor='loss', factor=0.6, patience=50, min_lr=1e-6)
callbacks=[lr_callback]

In [ ]:
history=model.fit(X_train,y_train,epochs=50,validation_data=(X_val,y_val))

TensorShape([243, 224, 224, 3])

Epoch 1/50
8/8 [==============================] - 18s 2s/step - loss: 0.5169 - r_square: 0.9915 - val_loss: 6.9358 - val_r_square: 0.7803
Epoch 2/50
8/8 [==============================] - 18s 2s/step - loss: 0.7140 - r_square: 0.9883 - val_loss: 5.9685 - val_r_square: 0.8110
Epoch 3/50
8/8 [==============================] - 20s 2s/step - loss: 0.4936 - r_square: 0.9919 - val_loss: 6.1473 - val_r_square: 0.8053
Epoch 4/50
8/8 [==============================] - 20s 2s/step - loss: 0.3930 - r_square: 0.9936 - val_loss: 6.7177 - val_r_square: 0.7873
Epoch 5/50
8/8 [==============================] - 16s 2s/step - loss: 0.3885 - r_square: 0.9936 - val_loss: 6.2478 - val_r_square: 0.8021
Epoch 6/50
8/8 [==============================] - 17s 2s/step - loss: 0.3912 - r_square: 0.9936 - val_loss: 6.7014 - val_r_square: 0.7878
Epoch 7/50
8/8 [==============================] - 18s 2s/step - loss: 0.3346 - r_square: 0.9945 - val_loss: 7.5166 - val_r_square: 0.7619
Epoch 8/50
8/8 [==================